In [1]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [2]:
IMG_SIZE = 224
# BATCH_SIZE = 32
EPOCHS = 20

MAX_SEQ_LENGTH = 100
NUM_FEATURES = 1920

In [3]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 9537
Total videos for testing: 3783


,video_name,tag
4510,v_JumpRope_g08_c02.avi,JumpRope
5477,v_PizzaTossing_g20_c04.avi,PizzaTossing
5252,v_Nunchucks_g08_c03.avi,Nunchucks
5488,v_PizzaTossing_g23_c03.avi,PizzaTossing
1964,v_CleanAndJerk_g14_c05.avi,CleanAndJerk
7841,v_SkyDiving_g15_c03.avi,SkyDiving
5310,v_Nunchucks_g18_c04.avi,Nunchucks
6956,v_Rafting_g25_c01.avi,Rafting
6944,v_Rafting_g22_c01.avi,Rafting
3866,v_HorseRace_g11_c05.avi,HorseRace


In [9]:
# The following two methods are taken from this tutorial:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)


In [10]:

def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet201(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input 

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [11]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'HammerThrow', 'Hammering', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpRope', 'JumpingJack', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'P

In [12]:
from tqdm import tqdm

def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in tqdm(enumerate(video_paths)):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


# train_data, train_labels = prepare_all_videos(train_df, "train")
test_data, test_labels = prepare_all_videos(test_df, "test")

# print(f"Frame features in train set: {train_data[0].shape}")
# print(f"Frame masks in train set: {train_data[1].shape}")

0it [00:00, ?it/s]2022-07-10 00:05:40.497486: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-07-10 00:05:41.463905: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-10 00:05:41.464543: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-10 00:05:41.464602: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-07-10 00:05:41.465321: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-10 00:05:41.465430: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
3783it [7:12:23,  6.86s/it]


In [14]:
train_data, train_labels = prepare_all_videos(train_df, "train")

9537it [18:34:16,  7.01s/it]


In [ ]:
# Utility for our sequence model.
# Bidirectional

def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5))(#16, 512
        frame_features_input, mask=mask_input
    )
#     x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(1024)(x)#8, 256
    x = keras.layers.LeakyReLU()(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    print(seq_model.summary())
    
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.2,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

In [ ]:
#40.13% LSTM(2048,dropout=0.1)
#46.47% LSTM(2048,dropout=0.5)
#43.96% LSTM(4096,dropout=0.5)

In [ ]:
class_vocab = label_processor.get_vocabulary()

frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# Refer to the following tutorial to understand the significance of using `mask`:
# https://keras.io/api/layers/recurrent_layers/gru/
x = keras.layers.LSTM(2048, return_sequences=True,dropout=0.1)(#16, 512
    frame_features_input, mask=mask_input
)
#     x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

rnn_model = keras.Model([frame_features_input, mask_input], output)

rnn_model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

print(rnn_model.summary())

In [ ]:
class_vocab = label_processor.get_vocabulary()

frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

# Refer to the following tutorial to understand the significance of using `mask`:
# https://keras.io/api/layers/recurrent_layers/gru/
x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.1)(#16, 512
    frame_features_input, mask=mask_input
)
# x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(1024, activation="relu")(x)#8, 256
output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

rnn_model = keras.Model([frame_features_input, mask_input], output)

rnn_model.compile(
    loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
)

print(rnn_model.summary())

In [ ]:

def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
def to_gif(images):
    converted_images = images.astype(np.uint8)
    imageio.mimsave("animation.gif", converted_images, fps=10)
    return embed.embed_file("animation.gif")


test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
to_gif(test_frames[:MAX_SEQ_LENGTH])

In [ ]:
ls -l

In [ ]:
import numpy as np
import tensorflow as tf 

np.save('Inception_CNNRNN_train_data.npy', train_data)    # .npy extension is added if not given
# np.save('Inception_CNNRNN_test_data.npy', test_data[0])    # .npy extension is added if not given
# np.save('Inception_CNNRNN_train_labels.npy', train_data[1])    # .npy extension is added if not given
# np.save('Inception_CNNRNN_test_labels.npy', test_labels)    # .npy extension is added if not given

In [ ]:
print(train_data[0].shape)
print(train_data[1].shape)

In [ ]:
print(train_data.type)

In [15]:
import pickle

with open('Dense201_CNNRNN_train_100.pickle', 'wb') as f:
    pickle.dump(train_data, f)

In [13]:
import pickle

with open('Dense201_CNNRNN_test_100.pickle', 'wb') as f:
    pickle.dump(test_data, f)

In [ ]:
import numpy as np
import tensorflow as tf 

# np.save('densenet_train_data.npy', train_data)    # .npy extension is added if not given
# np.save('densenet_test_data.npy', test_data)    # .npy extension is added if not given
np.save('CNNRNN_train_labels.npy', train_labels)    # .npy extension is added if not given
np.save('CNNRNN_test_labels.npy', test_labels)    # .npy extension is added if not given

In [ ]:
ls -l

In [ ]:
mv Inception_CNNRNN_20.pickle Inception_CNNRNN_train_20.pickle

In [4]:
import numpy as np
import tensorflow as tf 
import pickle

with open('Dense201_CNNRNN_train_100.pickle', 'rb') as f:
     d_train_data = pickle.load(f)
with open('Dense201_CNNRNN_test_100.pickle', 'rb') as f:
     d_test_data = pickle.load(f)
        
# d_train_data = np.load('densenet_train_data.npy')
# d_test_data = np.load('densenet_test_data.npy')
d_train_labels = np.load('CNNRNN_train_labels.npy')
d_test_labels = np.load('CNNRNN_test_labels.npy')

In [ ]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
# print(label_processor.get_vocabulary())

def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
        frame_features_input, mask=mask_input
    )    
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
#         frame_features_input, mask=mask_input
#     )
#     x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(1024)(x)#8, 256
    x = keras.layers.GaussianNoise(0.4)(x)
    x = keras.layers.LeakyReLU(0.1)(x)    
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
    return rnn_model

# Utility for running experiments.
def d_run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    print(seq_model.summary())
    
    optimizer = keras.optimizers.SGD(lr=1e-3)
#     optimizer = keras.optimizers.Adadelta()
    
    seq_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )    
    
    history = seq_model.fit(
        [d_train_data[0], d_train_data[1]],
        d_train_labels,
#         validation_split=0.2,
        validation_data=([d_test_data[0], d_test_data[1]],
        d_test_labels),
        epochs=100,
#         epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


d_run_experiment()

In [ ]:
#lr=0.1 78.24%
#lr=0.01 79.20%
#lr=0.001 77.98

In [ ]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
# print(label_processor.get_vocabulary())

def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
        frame_features_input, mask=mask_input
    )    
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
#         frame_features_input, mask=mask_input
#     )
#     x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(1024)(x)#8, 256
    x = keras.layers.GaussianNoise(0.4)(x)
    x = keras.layers.LeakyReLU(0.1)(x)    
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
    return rnn_model

# Utility for running experiments.
def d_run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    print(seq_model.summary())
    

    lr_schedule = keras.callbacks.LearningRateScheduler(
                  lambda epoch: 1e-6 * 10**(4*epoch / 10))
    
    optimizer = keras.optimizers.SGD(lr=1e-3)
    
    seq_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )    
#     seq_model.compile(optimizer=optimizer,
#                       loss='categorical_crossentropy',
#                      metrics=['accuracy'])     
    
    history = seq_model.fit(
        [d_train_data[0], d_train_data[1]],
        d_train_labels,
#         validation_split=0.2,
        validation_data=([d_test_data[0], d_test_data[1]],
        d_test_labels),
        epochs=20,
#         epochs=EPOCHS,
        callbacks=[lr_schedule,checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([d_test_data[0], d_test_data[1]], d_test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


his, seq = d_run_experiment()


In [ ]:
from matplotlib import pyplot as plt

plt.semilogx(his.history['lr'], his.history['accuracy'])
plt.axis([1e-6, 1, 0, 1])
plt.xlabel('lr')
plt.ylabel('accuracy')
plt.show()

In [ ]:
print(d_train_data[0].shape)
print(d_train_data[1].shape)

In [ ]:
print(d_train_data[1][0][39])

In [21]:
import numpy as np

# set length of video sequence and indices
MAX_SEQ_LENGTH = 20#override max sequence length
#max_seq=40
# copy_indices=[0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38]
# copy_indices=[0,4,8,12,16,20,24,28,32,36]
# copy_indices=[0,5,10,15,20,25,30,35]
# copy_indices=[0,8,16,24,32]
# copy_indices=[0,19]
# copy_indices=[0]
# max_seq=100
# # copy_indices=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
#               26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
#               51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,
#               76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]
copy_indices=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
# copy_indices=[0,10,20,30,40,50,60,70,80,90]
# copy_indices=[0,20,40,60,80]
# copy_indices=[0,25,50,75]
# copy_indices=[0,50]
d_train_data2_1 = np.copy(d_train_data[0][:,copy_indices, ])
d_train_data2_2 = np.copy(d_train_data[1][:,copy_indices, ])
d_train_data2 = (d_train_data2_1,d_train_data2_2)

d_test_data2_1 = np.copy(d_test_data[0][:,copy_indices, ])
d_test_data2_2 = np.copy(d_test_data[1][:,copy_indices, ])
d_test_data2 = (d_test_data2_1,d_test_data2_2)
# print(d_train_data2.shape)

label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
# print(label_processor.get_vocabulary())

def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.LSTM(2048, return_sequences=False,dropout=0.5)(#16, 512
        frame_features_input, mask=mask_input
    )    
#     x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False,dropout=0.5),merge_mode='concat')(#16, 512
#         frame_features_input, mask=mask_input
#     )
#     x = keras.layers.LSTM(2048)(x)#8, 256
#     x = keras.layers.Dropout(0.1)(x)
    x = keras.layers.Dense(1024)(x)#8, 256
    x = keras.layers.GaussianNoise(0.4)(x)
    x = keras.layers.LeakyReLU(0.1)(x)    
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

#     rnn_model.compile(
#         loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
#     )
    return rnn_model

# Utility for running experiments.
def d_run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    print(seq_model.summary())
    
    optimizer = keras.optimizers.SGD(lr=1e-2)
#     optimizer = keras.optimizers.Adadelta()
    
    seq_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )    
    
    history = seq_model.fit(
        [d_train_data2[0], d_train_data2[1]],
        d_train_labels,
#         validation_split=0.2,
        validation_data=([d_test_data2[0], d_test_data2[1]],
        d_test_labels),
        epochs=100,
#         epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([d_test_data2[0], d_test_data2[1]], d_test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


d_run_experiment()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 20, 1920)]   0           []                               
                                                                                                  
 input_14 (InputLayer)          [(None, 20)]         0           []                               
                                                                                                  
 lstm_4 (LSTM)                  (None, 2048)         32514048    ['input_13[0][0]',               
                                                                  'input_14[0][0]']               
                                                                                                  
 dense_8 (Dense)                (None, 1024)         2098176     ['lstm_4[0][0]']           

ValueError: Shapes (1024, 101) and (2048, 101) are incompatible

In [ ]:
#max seq = 40
#seq 1 75.20%
#seq 2 79.04%
#seq 5 80.33%
#seq 8 80.44%
#seq 10 79.75%
#seq 20 79.78%

#max seq = 100
#seq 2 79.91
#seq 4 81.52
#seq 5 81.95
#seq 10 81.52
#seq 20 81.13

In [4]:
import numpy as np
import tensorflow as tf 
import pickle

with open('Dense201_CNNRNN_train_40.pickle', 'rb') as f:
     d_train_data = pickle.load(f)
with open('Dense201_CNNRNN_test_40.pickle', 'rb') as f:
     d_test_data = pickle.load(f)
        
# d_train_data = np.load('densenet_train_data.npy')
# d_test_data = np.load('densenet_test_data.npy')
d_train_labels = np.load('CNNRNN_train_labels.npy')
d_test_labels = np.load('CNNRNN_test_labels.npy')

In [5]:
train_labels = d_train_labels
test_labels = d_test_labels

In [17]:
import numpy as np

MAX_SEQ_LENGTH = 5#override max sequence length
#max_seq=40
# copy_indices=[0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38]
# copy_indices=[0,4,8,12,16,20,24,28,32,36]
# copy_indices=[0,5,10,15,20,25,30,35]
copy_indices=[0,8,16,24,32]
# copy_indices=[0,19]
# copy_indices=[0]
# max_seq=100
# # copy_indices=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,
#               26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,
#               51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,
#               76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99]
# copy_indices=[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
# copy_indices=[0,10,20,30,40,50,60,70,80,90]
#copy_indices=[0,20,40,60,80]
# copy_indices=[0,25,50,75]
# copy_indices=[0,50]
train_data = np.copy(d_train_data[0][:,copy_indices, ])
test_data = np.copy(d_test_data[0][:,copy_indices, ])
print(train_data.shape)

(9537, 5, 1920)


In [7]:

class PositionalEmbedding(keras.layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = keras.layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask


In [8]:

class TransformerEncoder(keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.3
        )
        self.dense_proj = keras.Sequential(
            [keras.layers.Dense(dense_dim, activation=tf.nn.gelu), keras.layers.Dense(embed_dim),]
        )
        self.layernorm_1 = keras.layers.LayerNormalization()
        self.layernorm_2 = keras.layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


In [20]:
# Label preprocessing with StringLookup.
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
)

def get_compiled_model():
    sequence_length = MAX_SEQ_LENGTH
    embed_dim = NUM_FEATURES
    dense_dim = 64
    num_heads = 4
    classes = len(label_processor.get_vocabulary())

    inputs = keras.Input(shape=(None, None))
    x = PositionalEmbedding(
        sequence_length, embed_dim, name="frame_position_embedding"
    )(inputs)
    x = TransformerEncoder(embed_dim, dense_dim, num_heads, name="transformer_layer")(x)
    x = keras.layers.GlobalMaxPooling1D()(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model


def run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    model = get_compiled_model()
    history = model.fit(
        train_data,
        train_labels,
        # validation_split=0.15,
        validation_data=([test_data, test_labels]),
        epochs=100,
        callbacks=[checkpoint],
    )
    
    print(f"Test best accuracy: {round(max(history.history['val_accuracy'])*100,2)}%")
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

#     model.load_weights(filepath)
#     _, accuracy = model.evaluate(test_data, test_labels)
#     print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return model

trained_model = run_experiment()

Epoch 1/100
298/299 [============================>.] - ETA: 0s - loss: 5.3880 - accuracy: 0.0601
Epoch 1: val_loss improved from inf to 2.82301, saving model to /tmp/video_classifier
299/299 [==============================] - 11s 36ms/step - loss: 5.3877 - accuracy: 0.0601 - val_loss: 2.8230 - val_accuracy: 0.3280
Epoch 2/100
299/299 [==============================] - ETA: 0s - loss: 1.6254 - accuracy: 0.5795
Epoch 2: val_loss improved from 2.82301 to 1.63263, saving model to /tmp/video_classifier
299/299 [==============================] - 10s 35ms/step - loss: 1.6254 - accuracy: 0.5795 - val_loss: 1.6326 - val_accuracy: 0.6265
Epoch 3/100
298/299 [============================>.] - ETA: 0s - loss: 0.7303 - accuracy: 0.7895
Epoch 3: val_loss improved from 1.63263 to 1.47450, saving model to /tmp/video_classifier
299/299 [==============================] - 11s 36ms/step - loss: 0.7305 - accuracy: 0.7895 - val_loss: 1.4745 - val_accuracy: 0.6479
Epoch 4/100
297/299 [=======================

Epoch 30/100
297/299 [============================>.] - ETA: 0s - loss: 0.0787 - accuracy: 0.9764
Epoch 30: val_loss did not improve from 1.41854
299/299 [==============================] - 8s 28ms/step - loss: 0.0796 - accuracy: 0.9760 - val_loss: 2.3719 - val_accuracy: 0.7090
Epoch 31/100
299/299 [==============================] - ETA: 0s - loss: 0.0879 - accuracy: 0.9745
Epoch 31: val_loss did not improve from 1.41854
299/299 [==============================] - 8s 27ms/step - loss: 0.0879 - accuracy: 0.9745 - val_loss: 2.2671 - val_accuracy: 0.7124
Epoch 32/100
299/299 [==============================] - ETA: 0s - loss: 0.0594 - accuracy: 0.9826
Epoch 32: val_loss did not improve from 1.41854
299/299 [==============================] - 8s 27ms/step - loss: 0.0594 - accuracy: 0.9826 - val_loss: 1.9511 - val_accuracy: 0.7224
Epoch 33/100
298/299 [============================>.] - ETA: 0s - loss: 0.0523 - accuracy: 0.9840
Epoch 33: val_loss did not improve from 1.41854
299/299 [===========

299/299 [==============================] - 8s 28ms/step - loss: 0.0259 - accuracy: 0.9920 - val_loss: 2.8965 - val_accuracy: 0.7357
Epoch 60/100
299/299 [==============================] - ETA: 0s - loss: 0.0282 - accuracy: 0.9920
Epoch 60: val_loss did not improve from 1.41854
299/299 [==============================] - 8s 28ms/step - loss: 0.0282 - accuracy: 0.9920 - val_loss: 2.7484 - val_accuracy: 0.7209
Epoch 61/100
299/299 [==============================] - ETA: 0s - loss: 0.0274 - accuracy: 0.9926
Epoch 61: val_loss did not improve from 1.41854
299/299 [==============================] - 8s 28ms/step - loss: 0.0274 - accuracy: 0.9926 - val_loss: 2.9586 - val_accuracy: 0.7346
Epoch 62/100
299/299 [==============================] - ETA: 0s - loss: 0.0409 - accuracy: 0.9897
Epoch 62: val_loss did not improve from 1.41854
299/299 [==============================] - 8s 27ms/step - loss: 0.0409 - accuracy: 0.9897 - val_loss: 2.6237 - val_accuracy: 0.7243
Epoch 63/100
297/299 [============

Epoch 89/100
299/299 [==============================] - ETA: 0s - loss: 0.0128 - accuracy: 0.9957
Epoch 89: val_loss did not improve from 1.41854
299/299 [==============================] - 8s 28ms/step - loss: 0.0128 - accuracy: 0.9957 - val_loss: 2.8743 - val_accuracy: 0.7457
Epoch 90/100
298/299 [============================>.] - ETA: 0s - loss: 0.0272 - accuracy: 0.9928
Epoch 90: val_loss did not improve from 1.41854
299/299 [==============================] - 8s 28ms/step - loss: 0.0272 - accuracy: 0.9928 - val_loss: 3.4700 - val_accuracy: 0.7349
Epoch 91/100
297/299 [============================>.] - ETA: 0s - loss: 0.0215 - accuracy: 0.9938
Epoch 91: val_loss did not improve from 1.41854
299/299 [==============================] - 8s 28ms/step - loss: 0.0222 - accuracy: 0.9937 - val_loss: 3.2800 - val_accuracy: 0.7396
Epoch 92/100
297/299 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.9958
Epoch 92: val_loss did not improve from 1.41854
299/299 [===========

In [ ]:
#total sequence = 40
#max_seq = 2
#head=4
#dim=64 74.94
#128 74.46
#1024 71.03
#32  74.65

#head=6
#32 74.39
#64 75.34
#16 73.38
#128 73.96

#max_seq = 5
#head=4
#dim=32 75.44
#128 74.89
#64 75.42

